**This notebook is used on the webscraped data collected by the notebook webScraper_MIDI.ipynb. Therefore, the MIDI folder architecture needs to be like:**<br>
midi-files<br>
folder with MP3-Code as name <br>
full | demo <br>
collected midi files for the relating song<br>

Notebook
Removal of
- all demo files which aren't needed because we also have a full file
- mvt (for movement in a classic music piece) files with several mvt files, only keep files of mvt==1 then
- folders named 'full' but with keeping the content (simple architecture design question)

User has option to listen into relating youtube tracks to check for the quality of a MIDI file. There is the option to delete it.

In [150]:
import glob
import pandas as pd
import numpy as np
import shutil 
import os
import magic 
import webbrowser
import time

In [136]:
# !pip install python-magic==0.4.27

In [137]:
import sys
sys.path.insert(0, '../../stage-1/overall_used_tools')
import requirements_check as rc

rc.check(sys, [glob, pd, np, shutil, os, magic, webbrowser, time])

For 
glob 
python version needs to fit 
-> original version 3.7.3 
-> your version 3.7.3 (default, Mar 27 2019, 22:11:17) 
[GCC 7.3.0]


Your version identical with original version 
-> pandas 
-> 1.3.5


Your version identical with original version 
-> numpy 
-> 1.21.6


Possibly different versions: 
-> shutil 
-> original version 1.0.0 
-> your version  is not clearly visible. Go to your used python folder path .../python3.7/site-packages to investigate the version of your package. Also possible: have a look at the shell commands "pip show module_name" and "apt show module_name"


For 
os 
python version needs to fit 
-> original version 3.7.3 
-> your version 3.7.3 (default, Mar 27 2019, 22:11:17) 
[GCC 7.3.0]


Possibly different versions: 
-> magic 
-> original version 0.4.27 
-> your version <function version at 0x7f79afc08c80>


For 
webbrowser 
python version needs to fit 
-> original version 3.7.3 
-> your version 3.7.3 (default, Mar 27 2019, 22:11:17) 
[GCC 7.3.0]


Possibly 

### Having a look if there are also demo files:

In [151]:
path = 'midi-files/'
demo_path = path+'*/demo/'

In [139]:
filenames = glob.glob(demo_path + "*.mid")
filenames2 = glob.glob(demo_path + "*.MID")
all_demo_midi = filenames + filenames2
all_demo_midi = np.array(all_demo_midi)
all_demo_midi

array([], dtype=float64)

Are there besides the demo files also full versions?:

In [140]:
full_midi_codes = []
mp3_demo_codes = []
for file in all_demo_midi:
    
    file2_parts = file.split('/')
    mp3_code_demo = file2_parts[1]
    mp3_demo_codes.append(mp3_code_demo)
    
    full_path = file2_parts[0] + '/'+ file2_parts[1] + '/full/'
    full_file1 = glob.glob(full_path + '*mid')
    full_file2 = glob.glob(full_path + '*MID')    
    full_version_files = full_file1 + full_file2
    if full_version_files != []:
        mp3_code = full_version_files[0].split('/')[0]
        full_midi_codes.append(mp3_code)


Keeping only the songs for which we have a demo file and remove all other demo folders for which the samples have a full version:

In [141]:
demo_keeping = set(mp3_demo_codes) - set(full_midi_codes)
all_mp3 = glob.glob(path + '*') 

demo_mp3_delete = set(all_mp3) - demo_keeping
#demo_mp3_delete.remove('cleaner.ipynb')

In [142]:
for i in demo_mp3_delete:
    delete_path = i + '/demo'
    try:
        shutil.rmtree(delete_path) # remove not only folder but also all content in the folder
    except:
        pass

### Removing file types which are marked as midi but have 0 byte. Also remove unneeded text files:
Through small renaming bug during downloading already existing text files were accidently renamed to '.mid'.

In [143]:
all_files = glob.glob(path + '*/*/*.mid')
for single_file in all_files:
    true_file_type = magic.from_file(single_file)
    if true_file_type == 'empty': # the file is 0 bytes big
        os.remove(single_file)
        
all_txt_files = all_files = glob.glob(path + '*/*/webpageinfo.txt')
for single_txt in all_txt_files:
        os.remove(single_txt)


### Next, delete all movements of synphnys in which folders are several movents (!) and the movement is not One:

In [144]:
mvt_files1 = glob.glob(path + '*/*/*mvt*.mid')
mvt_files2 = glob.glob(path + '*/*/*MVT*.mid')
mvt_files3 = glob.glob(path + '*/*/*Mvt*.mid')
mvt_files = mvt_files1 + mvt_files2 + mvt_files3

# Check if the file is the only file in a certain directory:
for mvt_file in mvt_files:
    mvt_file_part = mvt_file.split('/')
    mvt_file_path = mvt_file_part[0] +  '/' +  mvt_file_part[1]+  '/' +  mvt_file_part[2]
    mvt_contained1 = glob.glob(mvt_file_path + '/*mvt*.mid')
    mvt_contained2 = glob.glob(mvt_file_path + '/*MVT*.mid')
    mvt_contained3 = glob.glob(mvt_file_path + '/*Mvt*.mid')
    mvt_contained = mvt_contained1 + mvt_contained2 + mvt_contained3
    if len(mvt_contained) > 0: # remove all not MVT1
        mvt_contained = np.array(mvt_contained)
        for mvt in mvt_contained:
            if 'one' not in mvt.lower() and '1' not in mvt.lower():
            
                os.remove(mvt)
                mvt_files.remove(mvt)
                

### Remove empty folders and the 'full' folders but with keeping of MIDI files

In [145]:
folder_list = os.listdir(path)
for single_folder in folder_list:
    if os.path.isdir(path + single_folder):
        longer_path = path + single_folder + '/*/*.mid'
        list_midi = glob.glob(longer_path)
        shorter_path = del_path = path + single_folder
        if len(list_midi ) == 0: # delete folder

            shutil.rmtree(shorter_path)

        else: 
            path_to_midi = list_midi[0]
            # move the midi to one folder before like command 'cd':
            shutil.move(path_to_midi,shorter_path)
            # remove the folder 'full':
            shutil.rmtree(shorter_path+'/full/')

### Open the explorer directories in which are several midi files. Wait until user gives input:

In [146]:
overall_path = path + '*/*/'
all_filenames = glob.glob(overall_path + "*.mid")
all_filenames2 = glob.glob(overall_path + "*.MID")
all_midi = np.array(all_filenames + all_filenames2)
len(all_midi)

0

In [147]:
# finding out the names of the single mp3 code samples:
path_info = '../../stage0_provided_information/gems-emotion-tags-main/data/GEMS_songs_overview.csv'
small = pd.read_csv(path_info, skiprows=1)
small2 = small[['Code','MP3-Code','Artist','Title']]
small3 = small2.dropna(how='all') # remove all rows only containing nan
small3

,Code,MP3-Code,Artist,Title
0,H001,H_Trailerpark_Schlech,Trailerpark,Schlechter Tag
1,H002,H_Desiigner_Panda,Desiigner,Panda
2,H003,H_NAS_DooRags,NAS,Doo Rags
3,H004,H_DMX_XGonGiv,DMX,X Gon' Give It To Ya
4,H005,H_Xatar_MeineGr,Xatar,Meine Große Liebe
...,...,...,...,...
365,P142,P_Oasis_DontLoo,Oasis,Don't Look Back In Anger
366,P143,P_DerJungemitderGitarre_HalloWo,Der Junge mit der Gitarre,Hallo Worum Gehts Ich Bin Dagegen
367,P144,P_ChrisBrown_Forever,Chris Brown,Forever
368,P145,P_RyanAdams_LuckyNo,NaN,Ryan Adams


In [148]:
for single_midi in all_midi:
           
    midi_parts = single_midi.split('/')
    before_path = midi_parts[0] + '/' + midi_parts[1] 
    before_path2 = before_path + '/' + midi_parts[2]
    
    number_ele = len(os.listdir(before_path2)) # count number of midi files for one sample
    
    if number_ele > 1: 
        # getting name sample:
        mp3_code = midi_parts[1]
        df_line = small3[small['MP3-Code']==str(mp3_code)]
        if len(df_line) == 0:
            df_line = small3[small['Code']==str(mp3_code)]
        title = df_line['Title'].to_string().split(' ')[1:]
        artist = df_line['Artist'].to_string().split(' ')[1:]
        print(' '.join(artist), ' '.join(title))
        
        time.sleep(3)
        
        webbrowser.open('https://www.youtube.com')
        time.sleep(15)
        print()
        webbrowser.open(before_path2)

        
        user_input = input('Next sample? [arbitrary answer for "yes"]')
    
        all_midi = [ele for ele in all_midi if not ele.startswith(before_path)]
        
    #else:
     #   all_midi.remove(single_midi)

### Give the folder names to the relating MIDI files and remove the folders

In [149]:
for path_midi in glob.glob(path +'*/*'):
    folder_mp3 = path_midi.split('/')[1]
    first_part = path_midi.split('/')[0]
    os.rename(path_midi, first_part + '/' + folder_mp3 + '.mid')
    os.rmdir(first_part + '/' + folder_mp3)

### Rename the folder in which the MIDI files are stored

In [152]:
os.rename(path, 'GeneratedMIDI_webscraping')